In [63]:
#création d'un premier model très simple (tfidf , stop word etc)

In [1]:
import pandas as pd

In [2]:
#récupération petit dataset (15k rows) d'entrainement et de test
#df = pd.read_csv('../set_train_simple/first_train_test15k.csv')
df = pd.read_csv('../set_train_simple/eqtype_train_test15k_.csv')
#division du set en 2 train

from sklearn.model_selection import train_test_split

train, test = train_test_split(df.copy(), test_size=(1/3), random_state=0)

In [3]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

In [5]:
df.type.value_counts()

people       3000
satirique    3000
actu         3000
science      3000
parodique    3000
Name: type, dtype: int64

In [68]:
train['titre']= train['titre'].astype(str)

In [69]:
liste_tok1 = []
from nltk import word_tokenize
for titre in train['titre']:
    words = word_tokenize(titre)
    liste_tok1.append(words)
train['tokken1']=liste_tok1

In [70]:
#nombre de tokken en moy par titre tok1
len([item for sublist in train['tokken1'] for item in sublist])/len(train)

11.7563

In [71]:
#taille du vocabulaire+ tok1
len(set([item for sublist in train['tokken1'] for item in sublist]))

21105

In [72]:
from nltk.tokenize import RegexpTokenizer
import re 

tokenizer = RegexpTokenizer(r'\w+') 

liste_tok2 = []
for titre in train['titre']:


    words = tokenizer.tokenize(re.sub('[0-9]+', '', titre.lower()))

    #new_words = list( set(words) - stop_words)   #=>
    new_words = [word for word in words if word not in stop_words]
    liste_tok2.append(new_words)
    
#imdb_data['tokkens'] = liste_tok
train['tokken2']=liste_tok2

In [73]:
#nombre de tokken en moy par titre tok2
len([item for sublist in train['tokken2'] for item in sublist])/len(train)

6.5277

In [74]:
#taille du vocabulaire tok2
len(set([item for sublist in train['tokken2'] for item in sublist]))

17103

In [93]:
vocabulaire = list(set([item for sublist in train['tokken2'] for item in sublist]))

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
#sur corpus train
corpus = list(train['titre'])
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulaire)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
#matrice entrainement
tab = pipe['count'].transform(corpus).toarray()
df_mod = pd.DataFrame(tab , columns = vocabulaire)
#gestion memoire
del pipe
del tab
del corpus

# #sur corpus test # vocabulaire reste celui du train 
# corpus_test = list(test['titre'])
# pipe_test = Pipeline([('count', CountVectorizer(vocabulary=vocabulaire)),
#                  ('tfid', TfidfTransformer())]).fit(corpus_test)
# #matrice test
# tab_test = pipe['count'].transform(corpus).toarray()
# df_mod_test = pd.DataFrame(tab_test , columns = vocabulaire)
# #gestion memoire
# del pipe_test
# del tab_test
# del corpus_test

In [108]:
df_mod['ethnique']

0       0
1       0
2       0
3       0
4       0
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: ethnique, Length: 10000, dtype: int64

In [76]:
#
tab = pipe['count'].transform(corpus).toarray()
df_mod = pd.DataFrame(tab , columns = vocabulaire)

In [77]:
df_mod

,vittel,insuffisances,atteinte,iraniens,autoconférence,fake,djokovic,klimt,complexée,informations,...,ep,religions,mosaïques,covit,bastille,elysée,infos,brunes,absurdes,proférer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
train['type'].value_counts()

satirique    2023
science      2019
people       2006
parodique    1986
actu         1966
Name: type, dtype: int64

In [79]:
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder().fit(train['type'])

In [80]:
encoder.classes_

array(['actu', 'parodique', 'people', 'satirique', 'science'],
      dtype=object)

In [81]:
encoder.inverse_transform([0,1,2,3,4])

array(['actu', 'parodique', 'people', 'satirique', 'science'],
      dtype=object)

In [82]:
Y=encoder.transform(train['type'])

In [83]:
from sklearn.ensemble import RandomForestClassifier

X=df_mod

clf = RandomForestClassifier(max_depth=5, random_state=8)
clf.fit(X, Y)

RandomForestClassifier(max_depth=5, random_state=8)

In [84]:
y_pred_train = clf.predict(X)

In [85]:
from sklearn.metrics import confusion_matrix
cm_train = confusion_matrix(Y, y_pred_train)

In [86]:
from sklearn import metrics

accuracy = metrics.accuracy_score(Y,y_pred_train)
print('Accuracy:',accuracy)

Accuracy: 0.4707


In [87]:
cm_train

array([[ 131,   93,   65, 1410,  267],
       [  28,  659,  125,  953,  221],
       [   1,   86, 1249,  590,   80],
       [   2,   45,   40, 1803,  133],
       [  11,   47,   29, 1067,  865]], dtype=int64)

In [88]:
from sklearn.naive_bayes import CategoricalNB

clf = CategoricalNB()
clf.fit(X, Y)


y_pred_train = clf.predict(X)

cm_train = confusion_matrix(Y, y_pred_train)

accuracy = metrics.accuracy_score(Y,y_pred_train)
print('Accuracy:',accuracy)

Accuracy: 0.8815


In [89]:
cm_train

array([[1461,   33,   30,  323,  119],
       [  47, 1684,   50,  134,   71],
       [  16,   68, 1856,   57,    9],
       [   5,    7,    4, 1988,   19],
       [  34,   13,    7,  139, 1826]], dtype=int64)

In [90]:
f1_score = metrics.f1_score(Y,y_pred_train, average='macro')
print('f1_score:',f1_score)

f1_score: 0.881356108341171


In [ ]:
#avec test
corpus = list(test['titre'])
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulaire)),
                 ('tfid', TfidfTransformer())]).fit(corpus)

tab = pipe['count'].transform(corpus).toarray()
df_mod = pd.DataFrame(tab , columns = vocabulaire)